In [1]:
import sklearn.metrics
import numpy as np
import pandas as pd
import gc

from transparentai.datasets import load_adult, load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from transparentai.models import classification

import transparentai.fairness as fairness

In [ ]:
pip install BlackBoxAuditing

In [2]:
data = load_adult()
X, Y = data.drop(columns='income'), data['income']
X = X.select_dtypes('number')
Y = Y.replace({'>50K':1, '<=50K':0})
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.33, random_state=42)
clf = RandomForestClassifier().fit(X_train,Y_train)

In [3]:
y_true = Y_train
y_true_valid = Y_valid
y_pred = clf.predict_proba(X_train)
y_pred_valid = clf.predict_proba(X_valid)

In [4]:
privileged_group = {
    'gender':['Male'],              
#     'marital-status': lambda x: 'Married' in x,
    'race':['White']
}

df_valid = data.loc[X_valid.index,:]
df_train = data.loc[X_train.index,:]

res_train1 = fairness.compute_fairness_metrics(y_true, 
                                     y_pred, 
                                     df_train,
                                     privileged_group)

res_valid1 = fairness.compute_fairness_metrics(y_true_valid, 
                                     y_pred_valid, 
                                     df_valid,
                                     privileged_group)

In [5]:
res_valid1

{'gender': {'statistical_parity_difference': -0.10176688863821445,
  'disparate_impact': 0.5629035343040948,
  'equal_opportunity_difference': -0.05801792869649808,
  'average_odds_difference': -0.0362636081490145,
  'theil_index': 0.14631697193948695},
 'race': {'statistical_parity_difference': -0.054154865761862986,
  'disparate_impact': 0.7387421874119033,
  'equal_opportunity_difference': -0.007496974897146402,
  'average_odds_difference': -0.006428405988372209,
  'theil_index': 0.14631697193948695}}

In [6]:
from aif360.algorithms import preprocessing

from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing import LFR, Reweighing
from aif360.algorithms.inprocessing import PrejudiceRemover
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing, EqOddsPostprocessing, RejectOptionClassification

In [27]:
from aif360.datasets import StandardDataset
from sklearn.preprocessing import LabelEncoder

def DataFrame_to_StandardDataset(df, target, privileged_group, favorable_label):
    """Converts a pandas.DataFrame into a 
    aif360.datasets.StandardDataset to use bias 
    processing algorithms.
    
    Parameters
    ----------
    df: pd.DataFrame
        DataFrame to convert
    target: str
        Column's name of the target
    privileged_group: dict
        Dictionnary with protected attribute as key (e.g. age or gender)
        and a list of favorable value (like ['Male']) or a function
        returning a boolean corresponding to a privileged group
    favorable_label: list or value
        The label of the favorable class. Needs to be in
        the df[target] column's unique value
        
    Returns
    -------
    aif360.datasets.StandardDataset:
        DataFrame converted
        
    Raises
    ------
    TypeError:
        df must be a pandas.DataFrame
    ValueError:
        target must be in df columns
    ValueError:
        At least one value of the favorable_label list needs to be in df[target]
    ValueError:
        favorable_label must be in df[target]
    """
    if type(df) != pd.DataFrame:
        raise TypeError('df must be a pandas.DataFrame')
    if target not in df.columns:
        raise ValueError('target must be in df columns')
    if (type(favorable_label) == list) & (
        not any([v in df[target].unique() for v in favorable_label])):
        raise ValueError('At least one value of the favorable_label list needs to be in df[target]')
    elif (type(favorable_label) != list):
        if (favorable_label not in df[target].unique()):
            raise ValueError('favorable_label must be in df[target]')
    
    df = df.copy()
    privileged_classes = [v for (k,v) in privileged_group.items()]
    
    if type(favorable_label) == list:
        df[target] = (df[target].isin(favorable_label)).astype(int)
    else:
        df[target] = (df[target] == favorable_label).astype(int)
    
    num_feats = df.select_dtypes('number').columns.tolist()
    
    protected_attribute_names = list(privileged_group.keys())
    privileged_classes = list()

    # Use Label Encoder for categorical columns (including target column)
    for attr, values in privileged_group.items():
        if attr in num_feats:
            privileged_classes.append(values)
            continue
        
        le = LabelEncoder()
        le.fit(df[attr])

        df[attr] = le.transform(df[attr])
        if type(values) == type(lambda x:x):
            fn = values
            tmp = [i for (i,v) in enumerate(le.classes_) if fn(v)]
            
        else:
            tmp = [np.where(le.classes_ == v)[0][0] for v in values]
        privileged_classes.append(tmp)
        
        num_feats += [attr]
    
    categorical_features = [c for c in df.columns if c not in num_feats]
    
    return StandardDataset(df=df, 
                   label_name=target, 
                   favorable_classes=[1],
                   categorical_features=categorical_features,
                   protected_attribute_names=protected_attribute_names,
                   privileged_classes=privileged_classes)
    

def get_priv_unpriv_grousp(dataset):
    """
    """
    unprivileged_groups = privileged_groups = list()
    
    for i, attr in enumerate(dataset.protected_attribute_names):
        unprivileged_groups.append({attr: dataset.unprivileged_protected_attributes[i]})
        privileged_groups.append({attr: dataset.privileged_protected_attributes[i]})
    
    return privileged_groups, unprivileged_groups


from aif360.algorithms.preprocessing import LFR, Reweighing, DisparateImpactRemover
preprocessing_algo = ['LFR','Reweighing','DisparateImpactRemover']
    
def mitigate_bias(df, target, privileged_group, favorable_label, model=None, algorithm=None):
    """
    """
    
    dataset = DataFrame_to_StandardDataset(df, target, 
                                           privileged_group, 
                                           favorable_label)
    
    privileged_groups, unprivileged_groups = get_priv_unpriv_grousp(dataset)
    
    dataset_transf = dataset.copy()
    
    if algorithm == 'Reweighing':
        obj = Reweighing(unprivileged_groups, privileged_groups)
        dataset_transf = obj.fit_transform(dataset_transf)
    
    elif algorithm == 'DisparateImpactRemover':        
        for attr in privileged_group:
            obj = DisparateImpactRemover(sensitive_attribute=attr)
            dataset_transf = obj.fit_transform(dataset_transf)
    
    elif algorithm == 'LFR':        
        for unpriv_group, priv_group in zip(unprivileged_groups, privileged_groups):
            obj = LFR([unpriv_group], [priv_group], k=1, verbose=0)
            dataset_transf = obj.fit_transform(dataset_transf)
        
    X = dataset_transf.features
    y = dataset_transf.labels.ravel()
    
    model.fit(X, y, sample_weight=dataset_transf.instance_weights)
    
    print('ok')
    return model

In [29]:
privileged_group = {
    'gender':['Male'], 
    'race':['White']
}

model = RandomForestClassifier(max_depth=10)
models = list()
for algo in preprocessing_algo:
    if algo =='LFR':
        continue
        
    print(algo)
    tmp = mitigate_bias(data, 'income', privileged_group, '>50K',
                        model=model, algorithm=algo)
    models.append(tmp)

Reweighing
ok
DisparateImpactRemover
ok


In [44]:
y_true = Y_train
y_true_valid = Y_valid

fairness_scores_train = list()
fairness_scores_valid = list()

df_valid = data.loc[X_valid.index,:]
df_train = data.loc[X_train.index,:]

df_train = DataFrame_to_StandardDataset(df_train, 'income', 
                                           privileged_group, 
                                           '>50K')
df_valid = DataFrame_to_StandardDataset(df_valid, 'income', 
                                           privileged_group, 
                                           '>50K')

for model in models:
    y_pred = model.predict_proba(df_train.features)
    y_pred_valid = model.predict_proba(df_valid.features)
    
    tmp = fairness.compute_fairness_metrics(y_true, 
                                     y_pred, 
                                     df_train.convert_to_dataframe()[0],
                                     privileged_group)
    
    fairness_scores_train.append(tmp)
    
    tmp = fairness.compute_fairness_metrics(y_true_valid, 
                                     y_pred_valid, 
                                     df_valid.convert_to_dataframe()[0],
                                     privileged_group)
    fairness_scores_valid.append(tmp)    

In [47]:
fairness_scores_train[0]

{'gender': {'statistical_parity_difference': -0.8313164649798312,
  'disparate_impact': 0.1686835350201687,
  'equal_opportunity_difference': -0.4512365250475586,
  'average_odds_difference': -0.7016036081496096,
  'theil_index': 0.12949214452620358},
 'race': {'statistical_parity_difference': -0.8313164649798312,
  'disparate_impact': 0.1686835350201687,
  'equal_opportunity_difference': -0.4512365250475586,
  'average_odds_difference': -0.7016036081496096,
  'theil_index': 0.12949214452620358}}

In [48]:
fairness_scores_train[1]

{'gender': {'statistical_parity_difference': -0.8313164649798312,
  'disparate_impact': 0.1686835350201687,
  'equal_opportunity_difference': -0.4512365250475586,
  'average_odds_difference': -0.7016036081496096,
  'theil_index': 0.12949214452620358},
 'race': {'statistical_parity_difference': -0.8313164649798312,
  'disparate_impact': 0.1686835350201687,
  'equal_opportunity_difference': -0.4512365250475586,
  'average_odds_difference': -0.7016036081496096,
  'theil_index': 0.12949214452620358}}

In [49]:
res_train1

{'gender': {'statistical_parity_difference': -0.1943527482593205,
  'disparate_impact': 0.36280897023389447,
  'equal_opportunity_difference': 0.002533869555253143,
  'average_odds_difference': 0.0010216595530061028,
  'theil_index': 0.001035250447756525},
 'race': {'statistical_parity_difference': -0.09475373448371352,
  'disparate_impact': 0.6269521232190262,
  'equal_opportunity_difference': 0.001269924973875347,
  'average_odds_difference': 0.00039545300236208436,
  'theil_index': 0.001035250447756525}}